# Lesson 3 - Building a RAG Agent with CrewAI

In this lesson, you will build a RAG agent with `CrewAI`. You will do that by integrating `RagTool` from `crewai_tools` with a `CrewAI` agent. `RagTool` provides a way to create and query knowledge bases from various data sources, and allows the agent to access specialized context. In this lesson, you will provide the RAG tool a pdf file containing details about insurance coverage provided by a private health insurer. By the end of the lesson, you will build an insurer agent specialized in answering queries related to health benefits. In the next lessons, you will wrap this agent in an ACP server and make it interact with other ACP agents.

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To Access <code>requirements.txt</code> and the <code>data</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

In [28]:
import sys
#!{sys.executable} -m pip install 'crewai[tools]'
#!{sys.executable} -m pip install ollama
!{sys.executable} -m pip install sentence-transformers

  Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 47.3 MB/s eta 0:00:0000:01
Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 53.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 52.9 MB/s eta 0:00:00
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 53.2 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: tokenizers╺━━━━━━━━━━━━━━━━━ 5/9 [scikit-learn]
    Found existing installation: to

## 3.1. Import Libraries

In [18]:
from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool

In [19]:
import warnings
warnings.filterwarnings('ignore')

## 3.2. Define the Agent's Large Language Model

You'll now define the large language model that you will use for your CrewAI agent. `max_tokens`: maximum number of tokens the model can generate in a single response.

**Note**: If you will define this model locally, it requires that you define the API key in a **.env** file as follows:
```
# Required
OPENAI_API_KEY=sk-...

# Optional
OPENAI_API_BASE=<custom-base-url>
OPENAI_ORGANIZATION=<your-org-id>
```

### crewai not integrate with llama4 yet

In [29]:
"""
llm = LLM(model="openai/gpt-4", max_tokens=1024)
llm = LLM(
    model="meta_llama/Llama-4-Scout-17B-16E-Instruct-FP8", 
    max_tokens=1024, 
    temperature=0.7
)
"""
#llm = LLM(model="openai/gpt-4o-mini", max_tokens=1024, temperature=0.7)
llm = LLM(model="ollama_chat/qwen2.5:14b", base_url="http://localhost:11434", max_tokens=8192)

## 3.3. Define the RAG Tool

For the RAG tool, you can define the model provider and the embedding model in a configuration Python dictionary. You can also define the details of your vector database. If you don't specify the vector database, the RagTool will use Chroma (ChromaDB) as the default vector database in local/in-memory mode.

In [37]:
config = {
    "llm": {
        "provider": "ollama",
        "config": {
            "model": "qwen2.5:14b",
        }
    },
    "embedding_model": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small"
        }
    }
}

You can then pass the `config` to the `RagTool`, and then specify the data source for which the knowledge base will be constructed. When embedding your data, the `RagTool` chunks your document into chunks and create an embedding vector for each chunk. You can specify the chunk size (`chunk_size`: number of characters) and how many characters overlap between consecutive chunks (`chunk_overlap`). You can also use the default behavior.

In [38]:
#from langchain.embeddings import HuggingFaceEmbeddings
#embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [39]:
rag_tool = RagTool(config=config,  
                   chunk_size=1200,       
                   chunk_overlap=200,     
                  )
rag_tool.add("./data/王梓蓉_論文改稿_v2.pdf", data_type="pdf_file")

2025-07-01 14:58:42,159 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 6 0 (offset 0)
2025-07-01 14:58:42,159 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 8 0 (offset 0)
2025-07-01 14:58:42,160 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 10 0 (offset 0)
2025-07-01 14:58:42,160 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 12 0 (offset 0)
2025-07-01 14:58:42,160 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 14 0 (offset 0)
2025-07-01 14:58:42,161 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 16 0 (offset 0)
2025-07-01 14:58:42,161 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 18 0 (offset 0)
2025-07-01 14:58:42,161 - 8486592256 - _utils.py-_utils:435 - WARNING: Ignoring wrong pointing object 20 0 (offset 0)
2025-07-01 14:58:42,162 - 8486592256 - _utils.py-_utils:43

In [40]:
rag_tool

RagTool(name='Knowledge base', description="Tool Name: Knowledge base\nTool Arguments: {'query': {'description': None, 'type': 'str'}}\nTool Description: A knowledge base that can be used to answer questions.", env_vars=[], args_schema=<class 'abc.RagToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x109267d00>, result_as_answer=False, max_usage_count=None, current_usage_count=0, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x168fe9780>, summarize=False), config={'llm': {'provider': 'ollama', 'config': {'model': 'qwen2.5:14b'}}, 'embedding_model': {'provider': 'openai', 'config': {'model': 'text-embedding-3-small'}}})

## 3.4. Define the Thesis Agent

Now that you have the `rag_tool` defined, you define the CrewAI agent that can assist with insurance coverage queries.

In [41]:
thesis_agent = Agent(
    role="Senior thesis qa Assistant", 
    goal="Understand the content of a thesis",
    backstory="You are an expert qa agent designed to assist with research queries for a particular thesis",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[rag_tool], 
    max_retry_limit=5
)

## 3.5. Define the Agent Task

Let's now test the insurance agent. For that, you need to define the agent task and pass to it the query and the agent.

In [42]:
task1 = Task(
        description="What is the contribution of the thesis?",
        expected_output = "A comprehensive response as to the users question",
        agent=thesis_agent
)

## 3.6. Run the Insurance Agent

To run the agent, you need to pass the agent and the task to a Crew object that you can run using the `kickoff` method.

In [43]:
crew = Crew(agents=[thesis_agent], tasks=[task1], verbose=True)
task_output = crew.kickoff()
print(task_output) 

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 49c7e79b-4357-45bd-8bab-6d71519df6df                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior thesis qa Assistant                                                                              │
│                                                                                                                 │
│  Task: What is the contribution of the thesis?                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior thesis qa Assistant                                                                              │
│                                                                                                                 │
│  Thought: I need to find out what the thesis contributes.                                                       │
│                                                                                                                 │
│  Using Tool: Knowledge base                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"What is the contribution of the thesis?\"}"                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  iii 致謝                                                                                                       │
│  還是到了寫致謝的時刻，這個瞬間已經在心裡重複無數次，卻不知如何下筆。這份論文的完成要感謝許多一路上幫助我的人   │
│  ，謝謝所有為我亮起的燈，在我失去方向的時候總是不吝嗇地伸出援手，謹在此致上最誠摯的感謝。                       │
│  謝謝爸爸、媽媽、哥哥、政佑，無論是在生活上的照顧還是情感上的支持，總是支撐著我的任何決定，讓我能做我自己，你   │
│  們始終是我最溫暖的避風港，也是我持續前行的最大動力。                                                           │
│  感謝陳毓鐸老師與蘇雅惠老師，在研究過程中給予我耐心指導與寶貴建議，不僅提升了我的學術視野，也培養我獨立思考與   │
│  實作的能力。也謝謝實驗室的學姊與同學，在論文與工作上各方面都給予許多幫助，讓這段過程少了很多手忙腳亂。         │
│  謝謝在這裡遇到的朋友，712、許光漢、實屬牛逼、窮國島、羽球系、suber                                             │
│  eat等數不清的群組，讓這兩年過得很充實，收穫了很多，難以形容的喜歡。謝謝大家總是互相幫忙，溫暖、熱情又接近吵鬧  │
│  ，我想過了很多年應該還會很懷念這段日子。                                                                       │
│  最後一位感謝留給我自己，感謝走得很慢卻一直在向前的自己、總是堅持想法的自己。儘管有很多事邊做邊後悔，去了很多   │
│  想去的地方、認識了很多人、有了很多回憶，也經歷了不少拉扯與調整。我們的人生很長，迷途中也要相信自己，冬天過去   │
│  春天便會到來。 我身在當時你幻想的未來裡，謝謝你一路陪我到這裡。 謹此致謝。 六月盡．114年夏                     │
│                                                                                                                 │
│  iii 致謝                                                                                                       │
│  還是到了寫致謝的時刻，這個瞬間已經在心裡重複無數次，卻不知如何下筆。這份論文的完成要感謝許多一路上幫助我的人   │
│  ，謝謝所有為我亮起的燈，在我失去方向的時候總是不吝嗇地伸出援手，謹在此致上最誠摯的感謝。                       │
│  謝謝爸爸、媽媽、哥哥、政佑，無論是在生活上的照顧還是情感上的支持，總是支撐著我的任何決定，讓我能做我自己，你   │
│  們始終是我最溫暖的避風港，也是我持續前行的最大動力。                                                           │
│  感謝陳毓鐸老師與蘇雅惠老師，在研究過程中給予我耐心指導與寶貴建議，不僅提升了我的學術視野，也培養我獨立思考與   │
│  實作的能力。也謝謝實驗室的學姊與同學，在論文與工作上各方面都給予許多幫助，讓這段過程少了很多手忙腳亂。         │
│  謝謝在這裡遇到的朋友，712、許光漢、實屬牛逼、窮國島、羽球系、suber                                             │
│  eat等數不清的群組，讓這兩年過得很充實，收穫了很多，難以形容的喜歡。謝謝大家總是互相幫忙，溫暖、熱情又接近吵鬧  │
│  ，我想過了很多年應該還會很懷念這段日子。                                                                       │
│  最後一位感謝留給我自己，感謝走得很慢卻一直在向前的自己、總是堅持想法的自己。儘管有很多事邊做邊後悔，去了很多   │
│  想去的地方、認識了很多人、有了很多回憶，也經歷了不少拉扯與調整。我們的人生很長，迷途中也要相信自己，冬天過去   │
│  春天便會到來。 我身在當時你幻想的未來裡，謝謝你一路陪我到這裡。 謹此致謝。 六月盡．114年夏                     │
│                                                                                                                 │
│  v 第四章 多模態問答系統之實驗設計與效能評估 . 29 4-1 實驗設計 . 29 4-2 實驗結果 . 35 4-3 結果分析與討論 . 44   │
│  4-4 實驗限制 . 49 第五章 結論 . 50 5-1 研究成果 . 50 5-2 實務意涵 . 50 5-3 研究限制 . 51 5-4 未來研究 . 53     │
│  參考文獻 . 54 附錄 . 58                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  v 第四章 多模態問答系統之實驗設計與效能評估 . 29 4-1 實驗設計 . 29 4-2 實驗結果 . 35 4-3 結果分析與討論 . 44   │
│  4-4 實驗限制 . 49 第五章 結論 . 50 5-1 研究成果 . 50 5-2 實務意涵 . 50 5-3 研究限制 . 51 5-4 未來研究 . 53     │
│  參考文獻 . 54 附錄 . 58                                                                                        │
│                                                                                                                 │
│  v 第四章 多模態問答系統之實驗設計與效能評估 . 29 4-1 實驗設計 . 29 4-2 實驗結果 . 35 4-3 結果分析與討論 . 44   │
│  4-4 實驗限制 . 49 第五章 結論 . 50 5-1 研究成果 . 50 5-2 實務意涵 . 50 5-3 研究限制 . 51 5-4 未來研究 . 53     │
│  參考文獻 . 54 附錄 . 58                                                                                        │
│                                                                                                                 │
│  7 1-4 研究重要性                                                                                               │
│  隨著寵物老齡化趨勢日益明顯，失智犬的照護議題逐漸成為飼主迫切關注的重要問題。然而，目前市面上或網路上的問答系   │
│  統大多僅依賴通用語料，未能有效整合專業獸醫文獻與實際社群照護經驗，也缺乏圖片與真實案例的輔助，導致提供的資訊   │
│  往往片面或不夠即時，無法滿足飼主與照護人員的實際需求。 本研究開發之多模態 RAG                                  │
│  專業問答系統，具體設計給飼主、獸醫師及寵物照護從業人員使用，透過結合 RAG                                       │
│  架構與多模態技術，有效整合專業獸醫文獻、中醫獸醫學教材及飼主社群討論，並以圖像與文字雙重支援的方式提供實際可   │
│  操作的建議。                                                                                                   │
│  在理論方面，本研究提供了檢索增強生成技術與多模態技術結合應用於專業領域問答系統的實證案例與效能驗證，對未來相   │
│  關技術研究具有重要的參考價值。在實務方面，透過此系統，飼主能迅速取得結合專業與實務經驗的完整資訊，獸醫師及寵   │
│  物照護從業人員也能更有效地掌握及傳遞專業知識與臨床經驗，有助於全面提升失智犬隻的生活品質與照護成效。此外，本   │
│  系統架構與方法亦可延伸至其他專業領域，作為開發類似專業輔助系統之實務參考範例。 1-5 論文架構                    │
│  本文的架構如下：第2章回顧相關文獻，聚焦於多模態與RAG在專業領域問答系統的應用。第3章說明本研究系統之整體架構與  │
│  實作流程，介紹系統框架與實現，涵蓋資料處理、圖文嵌入設計、檢索模組建置與提示詞設計。第4章描述實驗設計與評估指  │
│  標，包括實驗環境設置、評估流程，以及效能評估指標的詳細說明，以三組實驗對不同題型進行系統效能評估，並進一步分   │
│  析多階段優化策略對檢索與回答品質的影響和多模態的對於提升回應品質的效果。第5章呈現綜合分析，包含與基準模型的比  │
│  較分析、系統效能的詳細討論，總結主要研究成果，討論研究的限制，並提出未來研究的可能方向 (圖 2)。                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior thesis qa Assistant                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  本研究的貢獻在於開發了一個多模態 RAG                                                                           │
│  專業問答系統，該系統具體設計給飼主、獸醫師及寵物照護從業人員使用。透過結合 RAG                                 │
│  架構與多模態技術，有效整合專業獸醫文獻、中醫獸醫學教材及飼主社群討論，並以圖像與文字雙重支援的方式提供實際可   │
│  操作的建議。在理論方面，本研究提供了檢索增強生成技術與多模態技術結合應用於專業領域問答系統的實證案例與效能驗   │
│  證，對未來相關技術研究具有重要的參考價值。在實務方面，透過此系統，飼主能迅速取得結合專業與實務經驗的完整資訊   │
│  ，獸醫師及寵物照護從業人員也能更有效地掌握及傳遞專業知識與臨床經驗，有助於全面提升失智犬隻的生活品質與照護成   │
│  效。此外，本系統架構與方法亦可延伸至其他專業領域，作為開發類似專業輔助系統之實務參考範例。                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0250e883-2e5a-4b2b-acf5-cacc38a90c7f                                                                     │
│  Agent: Senior thesis qa Assistant                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 49c7e79b-4357-45bd-8bab-6d71519df6df                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 本研究的貢獻在於開發了一個多模態 RAG                                                             │
│  專業問答系統，該系統具體設計給飼主、獸醫師及寵物照護從業人員使用。透過結合 RAG                                 │
│  架構與多模態技術，有效整合專業獸醫文獻、中醫獸醫學教材及飼主社群討論，並以圖像與文字雙重支援的方式提供實際可   │
│  操作的建議。在理論方面，本研究提供了檢索增強生成技術與多模態技術結合應用於專業領域問答系統的實證案例與效能驗   │
│  證，對未來相關技術研究具有重要的參考價值。在實務方面，透過此系統，飼主能迅速取得結合專業與實務經驗的完整資訊   │
│  ，獸醫師及寵物照護從業人員也能更有效地掌握及傳遞專業知識與臨床經驗，有助於全面提升失智犬隻的生活品質與照護成   │
│  效。此外，本系統架構與方法亦可延伸至其他專業領域，作為開發類似專業輔助系統之實務參考範例。                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

本研究的貢獻在於開發了一個多模態 RAG 專業問答系統，該系統具體設計給飼主、獸醫師及寵物照護從業人員使用。透過結合 RAG 架構與多模態技術，有效整合專業獸醫文獻、中醫獸醫學教材及飼主社群討論，並以圖像與文字雙重支援的方式提供實際可操作的建議。在理論方面，本研究提供了檢索增強生成技術與多模態技術結合應用於專業領域問答系統的實證案例與效能驗證，對未來相關技術研究具有重要的參考價值。在實務方面，透過此系統，飼主能迅速取得結合專業與實務經驗的完整資訊，獸醫師及寵物照護從業人員也能更有效地掌握及傳遞專業知識與臨床經驗，有助於全面提升失智犬隻的生活品質與照護成效。此外，本系統架構與方法亦可延伸至其他專業領域，作為開發類似專業輔助系統之實務參考範例。


## 3.7. Resources

- [CrewAI Agents](https://docs.crewai.com/concepts/agents)
- [CrewAI Tasks](https://docs.crewai.com/concepts/tasks)
- [CrewAI RagTool](https://docs.crewai.com/tools/ai-ml/ragtool)
- [Short course on Multi Agents with CrewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>